# **Trabalho 2**
## Fernando Söndahl Brito

    DEPARTAMENTO DE ENGENHARIA DE PRODUÇÃO

    ANÁLISE DE EFICIÊNCIA PRODUTIVA

    PROFª. LÍDIA


## **Ajuste iniciais**

*   Pacotes e bibliotecas
*   Dados



> Instalação do framework e solver:

In [4]:
!apt-get install -y -qq glpk-utils
!pip install -q pyomo

In [5]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np

pd.options.mode.chained_assignment = None

> Conexão com os dados do problema:

In [6]:
dados = "https://github.com/fersondahl-uff/Trab2-BCC/raw/main/Dados%20barb.xlsx"

dados_df = pd.read_excel(io=dados, header=1)

dados_df = dados_df.rename(columns={'Unnamed: 0': 'DMU' ,'INPUT 1': 'Input 1',  'INPUT 2': 'Input 2',  'OUTPUT 1': 'Output 1',
                                    'OUTPUT 2': 'Output 2'})

dados_df

,DMU,Input 1,Input 2,Output 1,Output 2
0,CELPE,965.21,41.05,5.83,33.70
1,CEMAR,182.63,74.70,1.13,7.35
2,CEMIG,419.79,50.23,2.35,7.77
3,COELBA,281.13,27.22,1.44,8.46
4,COELCE,306.26,66.71,1.41,6.88
5,COPEL,701.51,112.61,2.21,14.61
6,CPFL PAULI.,627.12,197.27,2.13,8.36
7,CPFL PIRATIN.,300.62,81.76,1.55,5.52
8,ELEKTRO,317.21,118.39,2.82,9.12
9,ESCELSA,154.02,79.95,1.54,3.35


# **1.**

> Resolução do problema utilizando o Método BCC do envelope Orientado à 
*Inputs*

In [7]:
## Modelo BCC envolope orientado à input
resultado = {'DMU':[], 'h': [], 'lambda': [], 'folgas': []}
n_input = 2
n_output = 2

dmus = range(len(dados_df))
for problem in dmus:
    
    model = pyo.ConcreteModel()
    
    model.lambdas = pyo.Var(dmus, bounds=(0, np.inf))
    model.h_dmu = pyo.Var()
    
    model.slack = pyo.Var(range(n_input + n_output), bounds=(0, np.inf))
    
    
    lambdas = model.lambdas
    h_dmu = model.h_dmu
    folga = model.slack 
    
    model.C1 = pyo.ConstraintList()
    for j in range(1, n_input+1):
      model.C1.add(expr= sum(lambdas[i]*dados_df[f'Input {j}'][i] for i in dmus) - h_dmu * dados_df[f'Input {j}'][problem] + folga[j-1] ==0)
    
    
    if n_output == 1:
      model.C2 = pyo.Constraint(expr= sum(lambdas[i]*dados_df['Output'][i] for i in dmus) == dados_df['Output'][problem]  + folga[n_input] )
    else:
      model.C2 = pyo.ConstraintList()
      for j in range(1, n_output + 1):
        model.C2.add(expr=sum(lambdas[i]*dados_df[f'Output {j}'][i] for i in dmus) == dados_df[f'Output {j}'][problem]  + folga[n_input + j -1])
    
    model.C3 = pyo.Constraint(expr=sum(lambdas[i] for i in dmus) == 1)

    model.obj = pyo.Objective(expr=h_dmu, sense=minimize)
    

    opt = SolverFactory('glpk')
    opt.solve(model)

    
    lambdas_result = []
    for i in dmus:
        lambdas_result.append(pyo.value(lambdas[i]))
    
    s_result = []
    for s in range(n_input + n_output):
        s_result.append(pyo.value(folga[s]))    
    
    resultado['h'].append(pyo.value(h_dmu))
    resultado['lambda'].append(lambdas_result)
    resultado['folgas'].append(s_result)
    resultado['DMU'] = dados_df['DMU']

resultado_df = pd.DataFrame(data=resultado)

## Tratamento dos Resultados

lambda_list = []
column_name = []
for i in resultado_df.index:
    
    lambda_list.append(list(resultado_df.query(f'index == {i}')['lambda'].explode('lambda')))
    column_name.append(f'lambda_{i}')
    
lambdas_df = pd.DataFrame(lambda_list, columns=column_name)


folga_list = []
column_name = []
for i in resultado_df.index:
     folga_list.append(list(resultado_df.query(f'index == {i}')['folgas'].explode('folgas')))
     
for i in range(len(folga_list[0])):     
     column_name.append(f'folga_{i}')

  

resultado_df = pd.merge(left=resultado_df.drop(['lambda', 'folgas'], axis=1),
                        right=pd.merge(left=lambdas_df, right=pd.DataFrame(folga_list, columns=column_name), how='left', left_index=True, right_index=True),
                        how='left', left_index=True, right_index=True).set_index('DMU')

for i in dmus:
  resultado_df[f'lambda_{i}'] = resultado_df[f'lambda_{i}'].apply(lambda linha: round(linha, 4))

for j in range(1, n_output + n_input):
  resultado_df[f'folga_{j}'] = resultado_df[f'folga_{j}'].apply(lambda linha: round(linha, 4))

resultado_df

,h,lambda_0,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5,lambda_6,lambda_7,lambda_8,lambda_9,lambda_10,lambda_11,folga_0,folga_1,folga_2,folga_3
DMU,,,,,,,,,,,,,,,,,
CELPE,1.000000,1.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,-0.0000,0.0000
CEMAR,1.000000,0.0000,1.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,-0.0000,0.0000
CEMIG,0.907113,0.2006,0.0000,0.0,0.5041,0.0,0.0,0.0,0.0,0.0,0.2953,0.0,0.0000,0.0,0.0,0.0000,4.2433
COELBA,1.000000,0.0000,0.0000,0.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,-0.0000,-0.0000
COELCE,0.748459,0.0000,0.1778,0.0,0.5516,0.0,0.0,0.0,0.0,0.0,0.2706,0.0,0.0000,0.0,0.0,0.0019,0.0000
COPEL,0.570177,0.2744,0.6991,0.0,0.0265,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.2179,0.0000
CPFL PAULI.,0.425049,0.1090,0.2411,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.5727,0.0,0.0771,0.0,0.0,0.0000,0.0000
CPFL PIRATIN.,0.713526,0.0117,0.0000,0.0,0.4012,0.0,0.0,0.0,0.0,0.0,0.5871,0.0,0.0000,0.0,0.0,0.0000,0.2350
ELEKTRO,0.949133,0.0993,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.6031,0.0,0.2976,0.0,0.0,0.0000,0.0911


**A.** 

> Através do método do envelope orientado à *input*, a eficiência de cada DMU será o valor da função objetivo após a otimização, ou o próprio valor de h.

In [8]:
eficiencia_df = resultado_df[['h']].apply(lambda linha: round(linha*100, 2)).rename(columns={'h': 'Eficiência %'})
eficiencia_df

,Eficiência %
DMU,
CELPE,100.00
CEMAR,100.00
CEMIG,90.71
COELBA,100.00
COELCE,74.85
COPEL,57.02
CPFL PAULI.,42.50
CPFL PIRATIN.,71.35
ELEKTRO,94.91


---

**B.**

> Os lambdas definirão os *benchmark* de cada DMU, sendo caracterizados em 3 casos:


1.   A DMU é eficiente:
  
  Ela será seu próprio *Benchmark*, seu respectivo lambda igual a 1 e os outros iguais a zero;

2.   A DMU é fracamente eficiente:
  
  Por mais que sua eficiência seja igual a 100%, ela terá outra DMU como *benchmark*;

3.   A DMU é ineficiente:
  
  Terá outra DMU ou uma combinação linear de DMU's como *benchmark*.




In [9]:
lambda_dmu = dict(dados_df['DMU'])


bench_df = resultado_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']]

for i in range(12):
  bench_df[f'lambda_{i}'] = bench_df[f'lambda_{i}'].apply(lambda linha: lambda_dmu[i] if linha > 0 else '')

bench_df['benchmark'] = bench_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']].agg(' '.join, axis=1).apply(lambda linha: linha.strip())

bench_df = pd.merge(left=bench_df[['benchmark']], right=eficiencia_df, how='left', right_index=True, left_index=True).reset_index()
bench_df['status_dmu'] = np.where(bench_df['DMU'] == bench_df['benchmark'], 'Eficiente', np.where(bench_df['Eficiência %'] == 100, 'Fracamente Eficiente', 'Ineficiente'))

bench_df

,DMU,benchmark,Eficiência %,status_dmu
0,CELPE,CELPE,100.00,Eficiente
1,CEMAR,CEMAR,100.00,Eficiente
2,CEMIG,CELPE COELBA ESCELSA,90.71,Ineficiente
3,COELBA,COELBA,100.00,Eficiente
4,COELCE,CEMAR COELBA ESCELSA,74.85,Ineficiente
5,COPEL,CELPE CEMAR COELBA,57.02,Ineficiente
6,CPFL PAULI.,CELPE CEMAR ESCELSA RGE,42.50,Ineficiente
7,CPFL PIRATIN.,CELPE COELBA ESCELSA,71.35,Ineficiente
8,ELEKTRO,CELPE ESCELSA RGE,94.91,Ineficiente
9,ESCELSA,ESCELSA,100.00,Eficiente


> O cálculo do alvo:


In [10]:
alvos_df = pd.merge(left=dados_df.set_index('DMU'), right=resultado_df[['h', 'folga_0', 'folga_1', 'folga_2', 'folga_3']], how='left', right_index=True, left_index=True)

for i in range(1, n_input+1):
  alvos_df[f'alvo ^X{i}'] = (alvos_df[f'Input {i}']*alvos_df['h']) - alvos_df[f'folga_{i-1}']

for i in range(1, n_output+1):
  alvos_df[f'^Y{i}'] = alvos_df[f'Output {i}'] + alvos_df[f'folga_{n_input + i - 1}']

alvos_df

,Input 1,Input 2,Output 1,Output 2,h,folga_0,folga_1,folga_2,folga_3,alvo ^X1,alvo ^X2,^Y1,^Y2
DMU,,,,,,,,,,,,,
CELPE,965.21,41.05,5.83,33.70,1.000000,0.0,0.0,-0.0000,0.0000,965.210000,41.050000,5.8300,33.7000
CEMAR,182.63,74.70,1.13,7.35,1.000000,0.0,0.0,-0.0000,0.0000,182.630000,74.700000,1.1300,7.3500
CEMIG,419.79,50.23,2.35,7.77,0.907113,0.0,0.0,0.0000,4.2433,380.797081,45.564300,2.3500,12.0133
COELBA,281.13,27.22,1.44,8.46,1.000000,0.0,0.0,-0.0000,-0.0000,281.130000,27.220000,1.4400,8.4600
COELCE,306.26,66.71,1.41,6.88,0.748459,0.0,0.0,0.0019,0.0000,229.223179,49.929727,1.4119,6.8800
COPEL,701.51,112.61,2.21,14.61,0.570177,0.0,0.0,0.2179,0.0000,399.985042,64.207660,2.4279,14.6100
CPFL PAULI.,627.12,197.27,2.13,8.36,0.425049,0.0,0.0,0.0000,0.0000,266.556489,83.849341,2.1300,8.3600
CPFL PIRATIN.,300.62,81.76,1.55,5.52,0.713526,0.0,0.0,0.0000,0.2350,214.500139,58.337873,1.5500,5.7550
ELEKTRO,317.21,118.39,2.82,9.12,0.949133,0.0,0.0,0.0000,0.0911,301.074637,112.367915,2.8200,9.2111


---

**C.**

> Resolução do problema utilizando o Método BCC de multiplicadores Orientado à 
*Inputs*

In [11]:
## MODELO BCC MULTIPLICADORES ORIENTADO À INPUT

dmus = range(len(dados_df))

resultado = {"DMU": [], "v1": [], "v2": [], "u1": [], "u2": [], "eficiencia %": [], 'u_bcc': []}

for problem in dmus:
    
    
    model = pyo.ConcreteModel()
    
    model.inputs = pyo.Var(range(n_input), bounds=(0, np.inf))
    model.u = pyo.Var(range(n_output), bounds=(0, np.inf))
    u = model.u

    model.u_bcc = pyo.Var()
    u_bcc = model.u_bcc

    inputs = model.inputs
    
    model.C1 = pyo.Constraint(expr = dados_df['Input 1'][problem]*inputs[0] + dados_df['Input 2'][problem]*inputs[1] == 1 )
    
    model.C2 = pyo.ConstraintList()
    for dmu in dmus:
        model.C2.add(expr = u_bcc + sum(dados_df[f'Output {j}'][dmu]*u[j-1] for j in range(1, 1 + n_output)) - sum(dados_df[f'Input {i}'][dmu]*inputs[i-1] for i in range(1, n_input +1)) <= 0)
    
   
    model.obj = pyo.Objective(expr= sum(dados_df[f'Output {i}'][problem]*u[i-1] for i in range(1, 1 + n_output)) + u_bcc, sense=maximize)
    

    opt = SolverFactory('glpk')
    opt.solve(model)

    resultado['v1'].append(pyo.value(inputs[0]))
    resultado['v2'].append(pyo.value(inputs[1]))
    resultado['u1'].append(pyo.value(u[0]))
    resultado['u2'].append(pyo.value(u[1]))
    resultado['u_bcc'].append(pyo.value(u_bcc))
    resultado['DMU'] = dados_df['DMU']
    resultado['eficiencia %'].append(round(100*pyo.value(model.obj),2))
    

resultado_mult_input = pd.DataFrame(data=resultado).set_index('DMU')

resultado_mult_input['status'] = np.where(resultado_mult_input['eficiencia %'] == 100, 'Eficiente', 'Ineficiente')

print("Tabela com os resultados do método dos Multilplicadores (Orientação: Input)")
resultado_mult_input


Tabela com os resultados do método dos Multilplicadores (Orientação: Input)


,v1,v2,u1,u2,eficiencia %,u_bcc,status
DMU,,,,,,,
CELPE,0.000000,0.024361,0.000000,0.029674,100.00,0.000000,Eficiente
CEMAR,0.003034,0.005970,0.000000,0.013864,100.00,0.898099,Eficiente
CEMIG,0.001797,0.004892,0.295405,0.000000,90.71,0.212911,Ineficiente
COELBA,0.000000,0.036738,0.000000,0.000000,100.00,1.000000,Eficiente
COELCE,0.002286,0.004497,0.000000,0.010445,74.85,0.676599,Ineficiente
COPEL,0.001160,0.001651,0.000000,0.032357,57.02,0.097444,Ineficiente
CPFL PAULI.,0.001171,0.001346,0.094176,0.016262,42.50,0.088505,Ineficiente
CPFL PIRATIN.,0.001911,0.005203,0.314226,0.000000,71.35,0.226475,Ineficiente
ELEKTRO,0.001746,0.003768,0.296050,0.000000,94.91,0.114271,Ineficiente


---

# **2.**

> Resolução do problema utilizando o Método BCC do envelope Orientado à 
*Outputs*

In [12]:
## Modelo BCC envolope orientado à output
resultado = {'DMU':[], 'fi': [], 'lambda': [], 'folgas': []}
n_input = 2
n_output = 2

dmus = range(len(dados_df))
for problem in dmus:
    
    model = pyo.ConcreteModel()
    
    model.lambdas = pyo.Var(dmus, bounds=(0, np.inf))
    model.fi = pyo.Var()
    
    model.slack = pyo.Var(range(n_input + n_output), bounds=(0, np.inf))
    
    
    lambdas = model.lambdas
    fi = model.fi
    folga = model.slack 
    
    model.C1 = pyo.ConstraintList()
    for j in range(1, n_input+1):
      model.C1.add(expr= sum(lambdas[i]*dados_df[f'Input {j}'][i] for i in dmus) - dados_df[f'Input {j}'][problem] + folga[j-1] ==0)
    
    
    if n_output == 1:
      model.C2 = pyo.Constraint(expr= sum(lambdas[i]*dados_df['Output'][i] for i in dmus) - fi * dados_df['Output'][problem]  == folga[n_input] )
    else:
      model.C2 = pyo.ConstraintList()
      for j in range(1, n_output + 1):
        model.C2.add(expr=sum(lambdas[i]*dados_df[f'Output {j}'][i] for i in dmus) == fi* dados_df[f'Output {j}'][problem]  + folga[n_input + j -1])
    
    model.C3 = pyo.Constraint(expr=sum(lambdas[i] for i in dmus) == 1)

    model.obj = pyo.Objective(expr=fi, sense=maximize)
    

    opt = SolverFactory('glpk')
    opt.solve(model)

    
    lambdas_result = []
    for i in dmus:
        lambdas_result.append(pyo.value(lambdas[i]))
    
    s_result = []
    for s in range(n_input + n_output):
        s_result.append(pyo.value(folga[s]))    
    
    resultado['fi'].append(pyo.value(fi))
    resultado['lambda'].append(lambdas_result)
    resultado['folgas'].append(s_result)
    resultado['DMU'] = dados_df['DMU']

resultado_df_out = pd.DataFrame(data=resultado)

## Tratamento dos Resultados

lambda_list = []
column_name = []
for i in resultado_df_out.index:
    
    lambda_list.append(list(resultado_df_out.query(f'index == {i}')['lambda'].explode('lambda')))
    column_name.append(f'lambda_{i}')
    
lambdas_df = pd.DataFrame(lambda_list, columns=column_name)


folga_list = []
column_name = []
for i in resultado_df_out.index:
     folga_list.append(list(resultado_df_out.query(f'index == {i}')['folgas'].explode('folgas')))
     
for i in range(len(folga_list[0])):     
     column_name.append(f'folga_{i}')

  

resultado_df_out = pd.merge(left=resultado_df_out.drop(['lambda', 'folgas'], axis=1),
                        right=pd.merge(left=lambdas_df, right=pd.DataFrame(folga_list, columns=column_name), how='left', left_index=True, right_index=True),
                        how='left', left_index=True, right_index=True).set_index('DMU')

for i in dmus:
  resultado_df_out[f'lambda_{i}'] = resultado_df_out[f'lambda_{i}'].apply(lambda linha: round(linha, 4))

for j in range(n_output + n_input):
  resultado_df_out[f'folga_{j}'] = resultado_df_out[f'folga_{j}'].apply(lambda linha: round(linha, 4))

resultado_df_out

,fi,lambda_0,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5,lambda_6,lambda_7,lambda_8,lambda_9,lambda_10,lambda_11,folga_0,folga_1,folga_2,folga_3
DMU,,,,,,,,,,,,,,,,,
CELPE,1.000000,1.0000,-0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,-0.0000,0.0000
CEMAR,1.000000,-0.0000,1.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000
CEMIG,1.133803,0.2706,0.0000,0.0,0.3640,0.0,0.0,0.0,0.0,0.0,0.3654,0.0,0.0000,0.0,0.0000,0.0000,4.6129
COELBA,1.000000,-0.0000,-0.0000,0.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000
COELCE,1.469539,0.1599,0.3515,0.0,0.0981,0.0,0.0,0.0,0.0,0.0,0.3905,0.0,0.0000,0.0,0.0000,0.0000,0.0000
COPEL,1.698908,0.6630,0.3370,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,60.2212,0.4917,0.0000
CPFL PAULI.,2.353545,0.4247,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.5753,0.0,63.7035,0.0000,2.0693
CPFL PIRATIN.,1.566321,0.1624,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.7710,0.0,0.0667,0.0,0.0000,0.0000,0.2693
ELEKTRO,1.060928,0.1050,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.5461,0.0,0.3488,0.0,0.0000,0.0000,0.2009


**A.**

> As eficiências das DMU's mudam com o modelo orientado à *output*.


In [13]:
eficiencia_out_df = pd.DataFrame()
eficiencia_out_df['Eficiencia output %'] = resultado_df_out[['fi']].apply(lambda linha: round(100/linha, 2))
eficiencia_out_df = pd.merge(left=eficiencia_out_df, right=eficiencia_df, how='left', right_index=True, left_index=True)

eficiencia_out_df['Teste eficiencia'] = eficiencia_out_df['Eficiencia output %'] == eficiencia_out_df['Eficiência %']
eficiencia_out_df

,Eficiencia output %,Eficiência %,Teste eficiencia
DMU,,,
CELPE,100.00,100.00,True
CEMAR,100.00,100.00,True
CEMIG,88.20,90.71,False
COELBA,100.00,100.00,True
COELCE,68.05,74.85,False
COPEL,58.86,57.02,False
CPFL PAULI.,42.49,42.50,False
CPFL PIRATIN.,63.84,71.35,False
ELEKTRO,94.26,94.91,False


> O cálculo do alvo será feito da seguinte maneira para cada caso:


1.   *Inputs*:

  Os *inputs* iniciais serão somados com suas respectivas folgas;

2.   *Output*:

  O *output* inicial de cada DMU será multiplicado por seu fi e somado com a sua folga.


In [14]:
lambda_dmu = dict(dados_df['DMU'])

bench_out_df = resultado_df_out[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']]

for i in range(12):
  bench_out_df[f'lambda_{i}'] = bench_out_df[f'lambda_{i}'].apply(lambda linha: lambda_dmu[i] if linha > 0 else '')

bench_out_df['benchmark'] = bench_out_df[['lambda_0', 'lambda_1', 'lambda_2', 'lambda_3', 'lambda_4',
       'lambda_5', 'lambda_6', 'lambda_7', 'lambda_8', 'lambda_9', 'lambda_10',
       'lambda_11']].agg(' '.join, axis=1).apply(lambda linha: linha.strip())

bench_out_df = pd.merge(left=bench_out_df[['benchmark']], right=eficiencia_df, how='left', right_index=True, left_index=True).reset_index()
bench_out_df['status_dmu'] = np.where(bench_out_df['DMU'] == bench_out_df['benchmark'], 'Eficiente', np.where(bench_out_df['Eficiência %'] == 100, 'Fracamente Eficiente', 'Ineficiente'))

alvos = []
alvos_out = pd.merge(left=dados_df.set_index('DMU'), right=resultado_df_out[['fi', 'folga_0', 'folga_1', 'folga_2', 'folga_3']], how='left', right_index=True, left_index=True)

for i in range(1, n_input+1):
  alvos_out[f'alvo ^X{i}'] = alvos_out[f'Input {i}'] - alvos_out[f'folga_{i-1}']
  alvos.append(f'alvo ^X{i}')

for i in range(1, n_output+1):
  alvos_out[f'alvo ^Y{i}'] = alvos_out[f'Output {i}']*alvos_out['fi'] + alvos_out[f'folga_{n_input + i - 1}']
  alvos.append(f'alvo ^Y{i}')


bench_out_df = pd.merge(left=bench_out_df.set_index('DMU'), right=alvos_out[alvos], how='left', right_index=True, left_index=True)

bench_out_df

,benchmark,Eficiência %,status_dmu,alvo ^X1,alvo ^X2,alvo ^Y1,alvo ^Y2
DMU,,,,,,,
CELPE,CELPE,100.00,Eficiente,965.21,41.0500,5.830000,33.700000
CEMAR,CEMAR,100.00,Eficiente,182.63,74.7000,1.130000,7.350000
CEMIG,CELPE COELBA ESCELSA,90.71,Ineficiente,419.79,50.2300,2.664438,13.422553
COELBA,COELBA,100.00,Eficiente,281.13,27.2200,1.440000,8.460000
COELCE,CELPE CEMAR COELBA ESCELSA,74.85,Ineficiente,306.26,66.7100,2.072050,10.110429
COPEL,CELPE CEMAR,57.02,Ineficiente,701.51,52.3888,4.246286,24.821042
CPFL PAULI.,CELPE RGE,42.50,Ineficiente,627.12,133.5665,5.013052,21.744940
CPFL PIRATIN.,CELPE ESCELSA RGE,71.35,Ineficiente,300.62,81.7600,2.427798,8.915394
ELEKTRO,CELPE ESCELSA RGE,94.91,Ineficiente,317.21,118.3900,2.991817,9.876564


**B.**

**C.**

> Resolução do problema utilizando o Método BCC de multiplicadores Orientado à 
*outputs*

In [15]:
## MODELO BCC MULTIPLICADORES ORIENTADO À OUTPUT

dmus = range(len(dados_df))

resultado = {"DMU": [], "v1": [], "v2": [], "u1": [], "u2": [], "eficiencia %": [], 'v_bcc': []}

for problem in dmus:
    
    
    model = pyo.ConcreteModel()
    
    model.inputs = pyo.Var(range(n_input), bounds=(0, np.inf))
    model.u = pyo.Var(range(n_output),bounds=(0, np.inf))
    model.v_bcc = pyo.Var()


    u = model.u
    inputs = model.inputs
    v_bcc = model.v_bcc
    
    model.C1 = pyo.Constraint(expr = sum(dados_df[f'Output {i}'][problem]*u[i-1] for i in range(1, n_output +1)) == 1 )
    
    model.C2 = pyo.ConstraintList()
    for dmu in dmus:
        model.C2.add(expr= sum(dados_df[f'Output {j}'][dmu]*u[j-1] for j in range(1, n_output +1)) - sum(dados_df[f'Input {i}'][dmu]*inputs[i-1] for i in range(1, n_input +1)) - v_bcc<= 0)

    
    model.obj = pyo.Objective(expr= sum(dados_df[f'Input {i}'][problem]*inputs[i-1] for i in range(1, n_input +1)) + v_bcc, sense=minimize)
    

    opt = SolverFactory('glpk')
    opt.solve(model)

    resultado['v1'].append(pyo.value(inputs[0]))
    resultado['v2'].append(pyo.value(inputs[1]))
    resultado['u1'].append(pyo.value(u[0]))
    resultado['u2'].append(pyo.value(u[1]))
    resultado['v_bcc'].append(pyo.value(v_bcc))
    resultado['DMU'] = dados_df['DMU']
    resultado['eficiencia %'].append(round(100/pyo.value(model.obj), 2))
    

resultado_mult_output = pd.DataFrame(data=resultado).set_index('DMU')

resultado_mult_output['status'] = np.where(resultado_mult_output['eficiencia %'] == 100, 'Eficiente', 'Ineficiente')

print("Tabela com os resultados do método dos Multilplicadores (Orientação: Output)")
resultado_mult_input

resultado_mult_output

Tabela com os resultados do método dos Multilplicadores (Orientação: Output)


,v1,v2,u1,u2,eficiencia %,v_bcc,status
DMU,,,,,,,
CELPE,0.000000,0.000000,0.000000,0.029674,100.00,1.000000,Eficiente
CEMAR,0.004581,0.000000,0.000000,0.136054,100.00,0.163364,Eficiente
CEMIG,0.002588,0.007046,0.425532,0.000000,88.20,-0.306698,Ineficiente
COELBA,0.004239,0.006031,0.000000,0.118203,100.00,-0.355975,Eficiente
COELCE,0.004802,0.005892,0.386212,0.066198,68.05,-0.394141,Ineficiente
COPEL,0.002305,0.000000,0.000000,0.068446,58.86,0.082185,Ineficiente
CPFL PAULI.,0.001134,0.000000,0.469484,0.000000,42.49,1.642114,Ineficiente
CPFL PIRATIN.,0.003806,0.008210,0.645161,0.000000,63.84,-0.249023,Ineficiente
ELEKTRO,0.002092,0.004513,0.354610,0.000000,94.26,-0.136875,Ineficiente
